In [8]:
#AlexNet
import d2lzh as d2l
from mxnet import gluon,init,nd
from mxnet.gluon import data as gdata,nn
import os
import sys

net = nn.Sequential()
# 使⽤较⼤的11 x 11窗⼝来捕获物体。同时使⽤步幅4来较⼤幅度减⼩输出⾼和宽。这⾥使⽤的输出通
# 道数⽐LeNet中的也要⼤很多
net.add(nn.Conv2D(96,kernel_size=11,strides=4,activation='relu'),
       nn.MaxPool2D(pool_size=3,strides=2),
        # 减⼩卷积窗⼝，使⽤填充为2来使得输⼊与输出的⾼和宽⼀致，且增⼤输出通道数
       nn.Conv2D(256,kernel_size=5,padding=2,activation='relu'),
       nn.MaxPool2D(pool_size=3,strides=2),
        # 连续3个卷积层，且使⽤更⼩的卷积窗⼝。除了最后的卷积层外，进⼀步增⼤了输出通道数。
        # 前两个卷积层后不使⽤池化层来减⼩输⼊的⾼和宽
       nn.Conv2D(384,kernel_size=3,padding=1,activation='relu'),
       nn.Conv2D(384,kernel_size=3,padding =1,activation='relu'),
       nn.Conv2D(256,kernel_size=3,padding=1,activation='relu'),
       nn.MaxPool2D(pool_size=3,strides=2),
      # 这⾥全连接层的输出个数⽐LeNet中的⼤数倍。使⽤丢弃层来缓解过拟合
        nn.Dense(4096,activation='relu'),nn.Dropout(0.5),
       nn.Dense(4096,activation='relu'),nn.Dropout(0.5),
       nn.Dense(10))

In [9]:
X = nd.random.uniform(shape=(1,1,224,224))
net.initialize()
for layer in net:
    X = layer(X)
    print(layer.name,'output shape:\t',X.shape)

conv10 output shape:	 (1, 96, 54, 54)
pool6 output shape:	 (1, 96, 26, 26)
conv11 output shape:	 (1, 256, 26, 26)
pool7 output shape:	 (1, 256, 12, 12)
conv12 output shape:	 (1, 384, 12, 12)
conv13 output shape:	 (1, 384, 12, 12)
conv14 output shape:	 (1, 256, 12, 12)
pool8 output shape:	 (1, 256, 5, 5)
dense6 output shape:	 (1, 4096)
dropout4 output shape:	 (1, 4096)
dense7 output shape:	 (1, 4096)
dropout5 output shape:	 (1, 4096)
dense8 output shape:	 (1, 10)


In [10]:
def load_data_fashion_mnist(batch_size,resize=None,root = os.path.join('~','.mxnet','datasets','fashion-mnist')):
    root = os.path.expanduser(root)# 展开⽤⼾路径'~'
    transformer = []
    if resize:
        transformer +=[gdata.vision.transforms.Resize(resize)]
    transformer +=[gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)#串联transform
    mnist_train = gdata.vision.FashionMNIST(root = root,train = True)
    mnist_test = gdata.vision.FashionMNIST(root=root,train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 4
    train_iter = gdata.DataLoader(
        mnist_train.transform_first(transformer),batch_size,shuffle=True,
        num_workers=num_workers
    )
    test_iter = gdata.DataLoader(
        mnist_test.transform_first(transformer),batch_size,shuffle=False,
        num_workers=num_workers
    )
    return train_iter,test_iter
batch_size = 128
train_iter,test_iter = load_data_fashion_mnist(batch_size,resize=224)

In [12]:
lr,num_epochs,ctx = 0.01,5,d2l.try_gpu()
net.initialize(force_reinit=True,ctx=ctx,init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
d2l.train_ch5(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)

training on gpu(0)
epoch 1, loss 1.3302, train acc 0.509, test acc 0.754, time 131.3 sec
epoch 2, loss 0.6558, train acc 0.754, test acc 0.806, time 128.2 sec
epoch 3, loss 0.5436, train acc 0.798, test acc 0.826, time 129.6 sec


KeyboardInterrupt: 